In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
# Estandarización variables numéricas y Codificación variables categóricas
# ------------------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler
# Gestión datos desbalanceados
# ------------------------------------------------------------------------------
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
# Para separar los datos en train y test
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
#  Gestión de warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Pair programming Preprocesado
### Ana Gonzalez y Ana Campos

En esta lección realizaremos los cambios oportunos para poder ejecutar el modelo de regresión logística.
Cuando nos enfrentamos a problemas de regresión lineal vimos que eran necesarios algunos cambios antes de poder ajustar los modelos.

En el caso de la regresión logística lo tendremos que hacer.

Estos cambios incluyen:

- **Estandarización** de las variables predictoras numéricas
- **Codificación** de las variables categóricas
- **Balanceo** de la variable respuesta

In [ ]:
# cargamos el conjunto de datos que guardamos en la lección anterior
df_titanic = pd.read_csv("../data-log/.csv", index_col=0).reset_index(drop = True)
df_titanic.head()

In [ ]:
# nos hacemos una copia del df que acabamos de cargar

df = df_titanic.copy()

# Estandarización

In [ ]:
# iniciamos el método para escalar

scaler = StandardScaler()

In [ ]:
# recordemos que la estandarización solo se hace para las variables predictoras numéricas

numericas = df.select_dtypes(include = np.number)
numericas.head()

In [ ]:
# Si queremos eliminar alguna columna
numericas.drop(["survived", "pclass"], axis = 1, inplace = True)

In [ ]:
# ahora ya podemos ajustar nuestros datos.  

scaler.fit(numericas)

# transformamos los datos

X_escaladas = scaler.transform(numericas)

# por último convertiremos el array que nos devuelve en un dataframe. 

numericas_estandar = pd.DataFrame(X_escaladas, columns = numericas.columns)
numericas_estandar.head(2)

In [ ]:
# dropeamos las columnas sin estandarizar

df.drop(["age", "fare", "num_familiar"], axis = 1, inplace=True)
df.head()

In [ ]:
# lo unomos al dataframe original 

df = pd.concat([df, numericas_estandar], axis = 1)
# chequeamos que esta todo bien
df.head()

# Codificacion con datos ESTANDARIZADOS

In [ ]:
df.head(2)

¿ tienen orden nuestras variables? 

# No tienen

In [ ]:
lista_columnas = ["embarked", "maturity", "adult_male", "alone"]

df_encoded = pd.DataFrame()


for columna in lista_columnas:
    df_dummies = pd.get_dummies(df[columna], prefix_sep = "_", prefix = columna, dtype = int)

    df_encoded = pd.concat([df_encoded, df_dummies], axis = 1)



In [ ]:
# ya tenemos nuestro dataframe con las variables codificadas,
df_encoded.head()

In [ ]:
# el siguiente paso sería unir este dataframe con nuestro dataframe original para tener todos los datos en un mismo df

df_final = pd.concat([df, df_encoded], axis = 1)
df_final.head()

In [ ]:
# por último nos queda eliminar las columnas categóricas originales porque ya no nos hacen falta. 

df_final.drop(lista_columnas, axis = 1, inplace=True)
df_final.head(2)

# Tienen

In [ ]:
# definimos el diccionario

map_sex = {"male": 0, "female": 1}

In [ ]:
df_final["sex"] = df_final["sex"].map(map_sex)
df_final.head(2)

In [ ]:
# Dejo por si tenemos mas diccionariso que poner 

-------------------------------------------------------------

Vamos a aplicar esta codificación al *dataframe* original, el objetivo, tener dos datasets: 

- Uno con las variables categóricas codificadas y las numéricas estandarizadas.

- Unos con las variables categóricas codificadas y las numéricas sin estandarizar.  


Con estos dos csv ajustaremos nuestro modelo de regresión logística para comparar que modelo es mejor y como pueden cambiar las métricas. 

# Codificacion con datos SIN ESTANDARIZAR

In [ ]:
df.head(2)

# sin orden

In [ ]:
lista_columnas = ["embarked", "maturity", "adult_male", "alone"]

df_encoded2 = pd.DataFrame()


for columna in lista_columnas:
    df_dummies2 = pd.get_dummies(df_titanic[columna], prefix_sep = "_", prefix = columna, dtype = int)

    df_encoded2 = pd.concat([df_encoded2, df_dummies2], axis = 1)

In [ ]:
# ya tenemos nuestro dataframe con las variables codificadas,
df_encoded2.head()

In [ ]:
# el siguiente paso sería unir este dataframe con nuestro dataframe original para tener todos los datos en un mismo df

df_final2 = pd.concat([df_titanic, df_encoded2], axis = 1)
df_final2.head()

In [ ]:
# por último nos queda eliminar las columnas categóricas originales porque ya no nos hacen falta. 

df_final2.drop(lista_columnas, axis = 1, inplace=True)
df_final2.head(2)

# con orden

In [ ]:
# definimos el diccionario

map_sex = {"male": 0, "female": 1}

In [ ]:
df_final2["sex"] = df_final2["sex"].map(map_sex)
df_final2.head(2)

# Balanceo de nuestra variable respuesta

# Pandas

In [ ]:
# recordemos como estaban distribuidos nuestros datos

plt.figure(figsize=(8,5)) # para cambiar el tamaño de la figura

fig1 = sns.countplot(data = df_titanic, x = "survived",  color = "mediumaquamarine",  edgecolor='black')
fig1.set(xticklabels=["No", "Yes"]) 
plt.show(

## Downsampling

In [ ]:
# lo primero que hacemos es sacar el número de registros que tenemos para la clase minoritaria
num_minoritarios = df_final["survived"].value_counts()[1]
num_minoritarios

In [ ]:
# nos creamos un dataframe solo con las filas donde la variable respuesta sea Yes. 
minoritarios = df_final2[df_final["survived"] == 1]
minoritarios.head(2)

In [ ]:
# extraemos una muestra de la categoría mayoritaria que sea del mismo tamaño que la clase minotaria

mayoritarios = df_final[df_final["survived"] == 0].sample(num_minoritarios, random_state = 42)
mayoritarios.head(2)

In [ ]:
# Ahora es el momento de unir los *dataframes*
balanceado = pd.concat([minoritarios,mayoritarios],axis = 0)
balanceado.head(2)

In [ ]:
# chequeamos como es nuestro dataframe ahora
balanceado["survived"].value_counts()

---
El resultado es : 

### Upsampling

In [ ]:
num_mayoritarios = df_final["survived"].value_counts()[0]
num_mayoritarios

In [ ]:
# seleccionamos ls datos de la clase mayoritaria

mayoritarios2 = df_final[df_final["survived"]== 0]
mayoritarios2.head(2)

In [ ]:
# hacemos un selección aleatoria de datos de la clase minoritaria, para tener el mismo número que la clase mayoritaria

minoritarios2 =df_final[df_final["survived"]==1].sample(num_mayoritarios, replace=True)
minoritarios2.head(2)

In [ ]:
# unimos los dos dataframes

balanceado2 = pd.concat([mayoritarios2,minoritarios2], axis = 0)
balanceado2.head(2)

In [ ]:
# chequeamos como quedaron cada una de nuestras categorías

balanceado2["survived"].value_counts()

---
El resultado es : 